In [55]:
import sys
sys.path.append('../../')

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# First file

In [56]:
df1 = pd.read_csv("v1-start-1.csv")
df1 = fill_empty(df1, VColumns.v2(), False)
df1.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
0,0,,Domaine Févre - Chablis Premiére Cru Vaulorant,,,"0,75",2018,79.0,0.0,0.0,,
1,1,,Merseult Les Clous Villages Bouchard Pére et Fils,,,"0,75",2016,96.0,0.0,0.0,,
2,2,,Domaine de la Solitude Chateauneuf du Pape,,,"0,75",2017,71.0,0.0,0.0,,
3,3,,Chassagne Montrachet Lamy Pillot,,,"0,75",2014,104.0,0.0,0.0,,
4,4,,Chateau Grand Corbin Despagne Grand Cru,,,"0,75",2015,87.0,0.0,0.0,,


In [57]:
df1['size'] = "BOTTLE" # all are bottles
df1['price'] = df1['price'].apply(lambda x: int(x*100)) # convert to cents
df1.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
0,0,,Domaine Févre - Chablis Premiére Cru Vaulorant,,,BOTTLE,2018,7900,0.0,0.0,,
1,1,,Merseult Les Clous Villages Bouchard Pére et Fils,,,BOTTLE,2016,9600,0.0,0.0,,
2,2,,Domaine de la Solitude Chateauneuf du Pape,,,BOTTLE,2017,7100,0.0,0.0,,
3,3,,Chassagne Montrachet Lamy Pillot,,,BOTTLE,2014,10400,0.0,0.0,,
4,4,,Chateau Grand Corbin Despagne Grand Cru,,,BOTTLE,2015,8700,0.0,0.0,,


# Second File

## Read and clean

In [58]:
import re
with open("v1-start-2.txt", "r") as f:
    content = f.read()
print(content[:200])

TYPE_SPARKLING


PROSECCO CARMINA „LOGGIA“							0,75 l - 22 €


Sekt Castel Monreale Rosé	 Rametz Südtirol 					0,75 l – 45 €


Arunda Rosé Sekt – Südtirol							0,75 l - 48 €


Südtiroler Sekt Brut 


In [59]:
# clean up text

content = re.sub(r"\n{2,}", "\n\n", content) # remove multiple new lines
content = re.sub(r"\t+", " ", content) # remove multiple tabs
content = re.sub(r" +", " ", content) # remove multiple spaces
content = re.sub(r"–", "-", content) # fix dash
content = re.sub(r"0,75 ?[l|L]", "+ 0,75 l", content) # add plus sign to identify size
content = re.sub(r"0,375 ?[l|L]", "+ 0,375 l", content)

print(content)

TYPE_SPARKLING

PROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €

Sekt Castel Monreale Rosé Rametz Südtirol + 0,75 l - 45 €

Arunda Rosé Sekt - Südtirol + 0,75 l - 48 €

Südtiroler Sekt Brut Riserva „36“ Kellerei Meran + 0,75 l - 43 €
+ 1,5 l

Comitissa Pas Dosé Riserva - Südtirol + 0,75 l - 47 €
+ 1,5 l

FRANCIACORTA CA‘ DEL BOSCO + 0,75 l - 55 €
+ 1,5 l

Franciacorta Grande Cuvee „Bellavista“ + 0,75 l - 57 €
+ 1,5 l + 3 l + 6 l + 9 l

CHAMPAGNER LEGRAS & HAAS + 0,75 l - 69 €

CHAMPAGNER LEGRAS & HAAS + 0,375 l - 45 €

Ruinart Brut + 0,75 l - 105 €

Ruinart Brut Rose’ + 0,75 l - 130 €

CHAMPAGNER Dom Perignon VINTAGE + 0,75 l - 290 €

CHAMPAGNE DE VENOGE Princes De Blanc de Blanc + 0,75 l - 115 €

CHAMPAGNE DE VENOGE Princes Rose + 0,75 l - 120 €

CHAMPAGNE DE VENOGE Princes Blanc de Noirs + 0,75 l - 115 €

TYPE_ROSE

ROSALIE Cuvee KELLEREI MERAN + 0,75 l - 25 €

Merlot Rosé Meris St. Pauls + 0,75 l - 28 €

BLAUBURGUNDER ROSÉ - INNERLEITERHOF Schenna + 0,75 l - 30 €

MEROSE Rosé Cuvee Weingu

## Build dataframe

Split by type

In [60]:
content_l = content.split("TYPE_")[1:]
content_l

['SPARKLING\n\nPROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €\n\nSekt Castel Monreale Rosé Rametz Südtirol + 0,75 l - 45 €\n\nArunda Rosé Sekt - Südtirol + 0,75 l - 48 €\n\nSüdtiroler Sekt Brut Riserva „36“ Kellerei Meran + 0,75 l - 43 €\n+ 1,5 l\n\nComitissa Pas Dosé Riserva - Südtirol + 0,75 l - 47 €\n+ 1,5 l\n\nFRANCIACORTA CA‘ DEL BOSCO + 0,75 l - 55 €\n+ 1,5 l\n\nFranciacorta Grande Cuvee „Bellavista“ + 0,75 l - 57 €\n+ 1,5 l + 3 l + 6 l + 9 l\n\nCHAMPAGNER LEGRAS & HAAS + 0,75 l - 69 €\n\nCHAMPAGNER LEGRAS & HAAS + 0,375 l - 45 €\n\nRuinart Brut + 0,75 l - 105 €\n\nRuinart Brut Rose’ + 0,75 l - 130 €\n\nCHAMPAGNER Dom Perignon VINTAGE + 0,75 l - 290 €\n\nCHAMPAGNE DE VENOGE Princes De Blanc de Blanc + 0,75 l - 115 €\n\nCHAMPAGNE DE VENOGE Princes Rose + 0,75 l - 120 €\n\nCHAMPAGNE DE VENOGE Princes Blanc de Noirs + 0,75 l - 115 €\n\n',
 'ROSE\n\nROSALIE Cuvee KELLEREI MERAN + 0,75 l - 25 €\n\nMerlot Rosé Meris St. Pauls + 0,75 l - 28 €\n\nBLAUBURGUNDER ROSÉ - INNERLEITERHOF Schenna + 

In [61]:
wines_by_type = {}
for c in content_l:
    wines = c.split("\n\n")
    type = wines[0]
    wines = wines[1:-1]
    wines_by_type[type] = wines
wines_by_type

{'SPARKLING': ['PROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €',
  'Sekt Castel Monreale Rosé Rametz Südtirol + 0,75 l - 45 €',
  'Arunda Rosé Sekt - Südtirol + 0,75 l - 48 €',
  'Südtiroler Sekt Brut Riserva „36“ Kellerei Meran + 0,75 l - 43 €\n+ 1,5 l',
  'Comitissa Pas Dosé Riserva - Südtirol + 0,75 l - 47 €\n+ 1,5 l',
  'FRANCIACORTA CA‘ DEL BOSCO + 0,75 l - 55 €\n+ 1,5 l',
  'Franciacorta Grande Cuvee „Bellavista“ + 0,75 l - 57 €\n+ 1,5 l + 3 l + 6 l + 9 l',
  'CHAMPAGNER LEGRAS & HAAS + 0,75 l - 69 €',
  'CHAMPAGNER LEGRAS & HAAS + 0,375 l - 45 €',
  'Ruinart Brut + 0,75 l - 105 €',
  'Ruinart Brut Rose’ + 0,75 l - 130 €',
  'CHAMPAGNER Dom Perignon VINTAGE + 0,75 l - 290 €',
  'CHAMPAGNE DE VENOGE Princes De Blanc de Blanc + 0,75 l - 115 €',
  'CHAMPAGNE DE VENOGE Princes Rose + 0,75 l - 120 €',
  'CHAMPAGNE DE VENOGE Princes Blanc de Noirs + 0,75 l - 115 €'],
 'ROSE': ['ROSALIE Cuvee KELLEREI MERAN + 0,75 l - 25 €',
  'Merlot Rosé Meris St. Pauls + 0,75 l - 28 €',
  'BLAUBURGUNDER R

Build pairs of type and wine

In [62]:
wine_type_pairs = []
for type, wines in wines_by_type.items():
    for wine in wines:
        wine_type_pairs.append((type, wine))
wine_type_pairs[:10]

[('SPARKLING', 'PROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €'),
 ('SPARKLING', 'Sekt Castel Monreale Rosé Rametz Südtirol + 0,75 l - 45 €'),
 ('SPARKLING', 'Arunda Rosé Sekt - Südtirol + 0,75 l - 48 €'),
 ('SPARKLING',
  'Südtiroler Sekt Brut Riserva „36“ Kellerei Meran + 0,75 l - 43 €\n+ 1,5 l'),
 ('SPARKLING',
  'Comitissa Pas Dosé Riserva - Südtirol + 0,75 l - 47 €\n+ 1,5 l'),
 ('SPARKLING', 'FRANCIACORTA CA‘ DEL BOSCO + 0,75 l - 55 €\n+ 1,5 l'),
 ('SPARKLING',
  'Franciacorta Grande Cuvee „Bellavista“ + 0,75 l - 57 €\n+ 1,5 l + 3 l + 6 l + 9 l'),
 ('SPARKLING', 'CHAMPAGNER LEGRAS & HAAS + 0,75 l - 69 €'),
 ('SPARKLING', 'CHAMPAGNER LEGRAS & HAAS + 0,375 l - 45 €'),
 ('SPARKLING', 'Ruinart Brut + 0,75 l - 105 €')]

Build dataframe

In [63]:
df2 = pd.DataFrame(wine_type_pairs, columns=["section", "original_name"])
df2 = fill_empty(df2, ["section", "original_name"] + VColumns.v2(), False)
df2.head()

,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
0,SPARKLING,"PROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €",,,,,,,0,0,0,0,,
1,SPARKLING,"Sekt Castel Monreale Rosé Rametz Südtirol + 0,...",,,,,,,0,0,0,0,,
2,SPARKLING,"Arunda Rosé Sekt - Südtirol + 0,75 l - 48 €",,,,,,,0,0,0,0,,
3,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Mer...,,,,,,,0,0,0,0,,
4,SPARKLING,"Comitissa Pas Dosé Riserva - Südtirol + 0,75 l...",,,,,,,0,0,0,0,,


### Clean dataframe and fill empty columns

#### external_id

In [64]:
df2['external_id'] = list(range(len(df1), len(df1)+len(df2)))
df2.head()

,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
0,SPARKLING,"PROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €",17,,,,,,0,0,0,0,,
1,SPARKLING,"Sekt Castel Monreale Rosé Rametz Südtirol + 0,...",18,,,,,,0,0,0,0,,
2,SPARKLING,"Arunda Rosé Sekt - Südtirol + 0,75 l - 48 €",19,,,,,,0,0,0,0,,
3,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Mer...,20,,,,,,0,0,0,0,,
4,SPARKLING,"Comitissa Pas Dosé Riserva - Südtirol + 0,75 l...",21,,,,,,0,0,0,0,,


#### type

In [65]:
df2['type'] = df2['section'].apply(lambda x: x if x!="MAGNUM" else "")
display(df2.loc[df2['type']==""])
df2.head()

,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
147,MAGNUM,Südtiroler Sekt Brut Riserva „36“ Kellerei Mer...,164,,,,,,0,0,0,0,,
148,MAGNUM,“Comitissa” Pas Dosé Riserva - Südtirol DOC 96 €,165,,,,,,0,0,0,0,,
149,MAGNUM,“FRANCIACORTA” CA‘ DEL BOSCO 114 €,166,,,,,,0,0,0,0,,
150,MAGNUM,“LUGANA” - CA DEI FRATI 64 €,167,,,,,,0,0,0,0,,
151,MAGNUM,„Stoan“ Kellerei Tramin 84 €,168,,,,,,0,0,0,0,,
152,MAGNUM,Sylvaner „Alte Reben“ Pacherhof 94 €,169,,,,,,0,0,0,0,,
153,MAGNUM,SAUVIGNON “QUARZ” - TERLAN 170 €,170,,,,,,0,0,0,0,,
154,MAGNUM,pino grigio st. michael eppan 3 l,171,,,,,,0,0,0,0,,
155,MAGNUM,“Trattmann” Pinot Noir Riserva Kellerei Girlan...,172,,,,,,0,0,0,0,,
156,MAGNUM,Merlot Riserva “Passion Limited Edition” DOC 2...,173,,,,,,0,0,0,0,,


,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
0,SPARKLING,"PROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €",17,SPARKLING,,,,,0,0,0,0,,
1,SPARKLING,"Sekt Castel Monreale Rosé Rametz Südtirol + 0,...",18,SPARKLING,,,,,0,0,0,0,,
2,SPARKLING,"Arunda Rosé Sekt - Südtirol + 0,75 l - 48 €",19,SPARKLING,,,,,0,0,0,0,,
3,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Mer...,20,SPARKLING,,,,,0,0,0,0,,
4,SPARKLING,"Comitissa Pas Dosé Riserva - Südtirol + 0,75 l...",21,SPARKLING,,,,,0,0,0,0,,


#### name

In [66]:
name_split_regex = r"(\+ ?.{1,5} ?[l|L]|\d+ ?€|\n)"

df2['name'] = df2['original_name'].apply(lambda x: re.split(name_split_regex, x)[0])
display(df2.loc[df2['name'].isna() | df2['name']==""])
df2.head()

,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes


,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
0,SPARKLING,"PROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €",17,SPARKLING,PROSECCO CARMINA „LOGGIA“,,,,0,0,0,0,,
1,SPARKLING,"Sekt Castel Monreale Rosé Rametz Südtirol + 0,...",18,SPARKLING,Sekt Castel Monreale Rosé Rametz Südtirol,,,,0,0,0,0,,
2,SPARKLING,"Arunda Rosé Sekt - Südtirol + 0,75 l - 48 €",19,SPARKLING,Arunda Rosé Sekt - Südtirol,,,,0,0,0,0,,
3,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Mer...,20,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Meran,,,,0,0,0,0,,
4,SPARKLING,"Comitissa Pas Dosé Riserva - Südtirol + 0,75 l...",21,SPARKLING,Comitissa Pas Dosé Riserva - Südtirol,,,,0,0,0,0,,


#### winery_name

we do not use it here

#### info

In [67]:
size_regex = r"\+ ?.{1,5} ?[l|L]"
price_regex = r"\d+ ?€"
info_parentheses_regex = r"\(.*\)"

def get_info(x):
    # check if info is in parentheses
    parentheses_info = re.findall(info_parentheses_regex, x)
    if len(parentheses_info) > 0:
        info = " ".join(parentheses_info)
        return re.sub(r"\(|\)", "", info).strip()

    # use second line if it doesn't contain price or size
    l = x.split("\n")
    if len(l) == 1:
        return ""
    has_price = re.match(price_regex, l[1])
    has_size = re.match(size_regex, l[1])
    if has_price or has_size:
        return ""
    return "\n".join(l[1:]).strip()

df2['info'] = df2['original_name'].apply(get_info)

# remove info from name
df2['name'] = df2['name'].apply(lambda x: re.sub(info_parentheses_regex, "", x).strip())

df2.loc[df2['info'] != ""]

,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
19,ROSE,Lagrein Rose` - Pfannenstielhof (Sieger beim A...,36,ROSE,Lagrein Rose` - Pfannenstielhof,,Sieger beim Autochtona Award,,0,0,0,0,,
28,WHITE,Pinot Grigio “Punggl” Kellerei Nals Margreid +...,45,WHITE,Pinot Grigio “Punggl” Kellerei Nals Margreid,,Gambero Rosso 3 Bicchieri,,0,0,0,0,,
33,WHITE,Weissburgunder “Sirmian” Kellerei Nals Margrei...,50,WHITE,Weissburgunder “Sirmian” Kellerei Nals Margreid,,"Gambero Rosso 3 Bicchieri, James Suckling 95 P...",,0,0,0,0,,
37,WHITE,"SYLVANER „PRAEPOSITUS“ - NEUSTIFT + 0,75 l - 3...",54,WHITE,SYLVANER „PRAEPOSITUS“ - NEUSTIFT,,Gambero Rosso,,0,0,0,0,,
40,WHITE,RIESLING „VINSCHGAU“ - FALKENSTEIN (trebicchie...,57,WHITE,RIESLING „VINSCHGAU“ - FALKENSTEIN,,"trebicchieri, Gambero Rosso",,0,0,0,0,,
43,WHITE,"KERNER „PRAEPOSITUS“ - KLOSTER NEUSTIFT + 0,75...",60,WHITE,KERNER „PRAEPOSITUS“ - KLOSTER NEUSTIFT,,Wine Hunter Award Platinum & Gold,,0,0,0,0,,
44,WHITE,MÜLLER THURGAU „FELDMARSCHALL“ TIEFENBRUNNER +...,61,WHITE,MÜLLER THURGAU „FELDMARSCHALL“ TIEFENBRUNNER,,Trebicchieri Gambero Rosso,,0,0,0,0,,
45,WHITE,Chardonnay „Magred“ Kellerei Nals Margreid + 0...,62,WHITE,Chardonnay „Magred“ Kellerei Nals Margreid,,James Suckling 92 Punkte,,0,0,0,0,,
48,WHITE,Chardonnay Riserva „Baron Salvadori“ Kellerei ...,65,WHITE,Chardonnay Riserva „Baron Salvadori“ Kellerei ...,,James Suckling 95 Punkte,,0,0,0,0,,
61,WHITE,„Dolomytos“ Weisswein Cuvee Weingut Dolomytos ...,78,WHITE,„Dolomytos“ Weisswein Cuvee Weingut Dolomytos,,Cuvee aus Südtiroler und Griechischen Rebsorten,,0,0,0,0,,


#### size

In [68]:
size_regex = r"\+ ?.{1,5} ?[l|L]"

def get_size(x):
    size = tuple(re.findall(size_regex, x))
    if len(size) == 0:
        return ('+ 0,75 l',)
    return size

df2['size'] = df2['original_name'].apply(get_size)

# remove size from name and info
df2['name'] = df2['name'].apply(lambda x: re.sub(size_regex, "", x).strip())
df2['info'] = df2['info'].apply(lambda x: re.sub(size_regex, "", x).strip())

# handle MAGNUM sizes separately
size_regex_magnum = r"\d ?[l|L]" # size is written with a different pattern in the MAGNUM section
for i, row in df2.iterrows():
    if row['section'] != "MAGNUM":
        continue
    size = tuple(re.findall(size_regex_magnum, row['original_name']))
    if len(size) == 0:
        size = ('+ 1,5 l',)
    df2.at[i, 'size'] = size

    # remove size from name and info in magnum
    df2.at[i, 'name'] = re.sub(size_regex_magnum, "", row['name']).strip()
    df2.at[i, 'info'] = re.sub(size_regex_magnum, "", row['info']).strip()

display(df2.loc[df2['size'].apply(len)==0])
display(df2.head())
df2.loc[df2['section']=='MAGNUM']

,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes


,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
0,SPARKLING,"PROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €",17,SPARKLING,PROSECCO CARMINA „LOGGIA“,,,"(+ 0,75 l,)",0,0,0,0,,
1,SPARKLING,"Sekt Castel Monreale Rosé Rametz Südtirol + 0,...",18,SPARKLING,Sekt Castel Monreale Rosé Rametz Südtirol,,,"(+ 0,75 l,)",0,0,0,0,,
2,SPARKLING,"Arunda Rosé Sekt - Südtirol + 0,75 l - 48 €",19,SPARKLING,Arunda Rosé Sekt - Südtirol,,,"(+ 0,75 l,)",0,0,0,0,,
3,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Mer...,20,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Meran,,,"(+ 0,75 l, + 1,5 l)",0,0,0,0,,
4,SPARKLING,"Comitissa Pas Dosé Riserva - Südtirol + 0,75 l...",21,SPARKLING,Comitissa Pas Dosé Riserva - Südtirol,,,"(+ 0,75 l, + 1,5 l)",0,0,0,0,,


,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
147,MAGNUM,Südtiroler Sekt Brut Riserva „36“ Kellerei Mer...,164,,Südtiroler Sekt Brut Riserva „36“ Kellerei Meran,,,"(+ 1,5 l,)",0,0,0,0,,
148,MAGNUM,“Comitissa” Pas Dosé Riserva - Südtirol DOC 96 €,165,,“Comitissa” Pas Dosé Riserva - Südtirol DOC,,,"(+ 1,5 l,)",0,0,0,0,,
149,MAGNUM,“FRANCIACORTA” CA‘ DEL BOSCO 114 €,166,,“FRANCIACORTA” CA‘ DEL BOSCO,,,"(+ 1,5 l,)",0,0,0,0,,
150,MAGNUM,“LUGANA” - CA DEI FRATI 64 €,167,,“LUGANA” - CA DEI FRATI,,,"(+ 1,5 l,)",0,0,0,0,,
151,MAGNUM,„Stoan“ Kellerei Tramin 84 €,168,,„Stoan“ Kellerei Tramin,,,"(+ 1,5 l,)",0,0,0,0,,
152,MAGNUM,Sylvaner „Alte Reben“ Pacherhof 94 €,169,,Sylvaner „Alte Reben“ Pacherhof,,,"(+ 1,5 l,)",0,0,0,0,,
153,MAGNUM,SAUVIGNON “QUARZ” - TERLAN 170 €,170,,SAUVIGNON “QUARZ” - TERLAN,,,"(+ 1,5 l,)",0,0,0,0,,
154,MAGNUM,pino grigio st. michael eppan 3 l,171,,pino grigio st. michael eppan,,,"(3 l,)",0,0,0,0,,
155,MAGNUM,“Trattmann” Pinot Noir Riserva Kellerei Girlan...,172,,“Trattmann” Pinot Noir Riserva Kellerei Girlan,,,"(+ 1,5 l,)",0,0,0,0,,
156,MAGNUM,Merlot Riserva “Passion Limited Edition” DOC 2...,173,,Merlot Riserva “Passion Limited Edition” DOC 2016,,,"(+ 1,5 l,)",0,0,0,0,,


#### vintage

In [69]:
vintage_regex = r"\d{4}"

def get_vintage(x):
    v = re.findall(vintage_regex, x)
    if len(v) == 0:
        return 0
    return v

df2['vintage'] = df2['original_name'].apply(get_vintage)

# remove vintage from name and info
df2['name'] = df2['name'].apply(lambda x: re.sub(vintage_regex, "", x).strip())
df2['info'] = df2['info'].apply(lambda x: re.sub(vintage_regex, "", x).strip())

df2.loc[df2['vintage'] != 0]

,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
36,WHITE,"Terlaner „Rarity“ 2010 + 0,75 l - 195 €",53,WHITE,Terlaner „Rarity“,,,"(+ 0,75 l,)",[2010],0,0,0,,
146,RED,SASSICAIA - TENUTA SAN GUIDO 2004 2011 2012 20...,163,RED,SASSICAIA - TENUTA SAN GUIDO,,,"(+ 0,75 l,)","[2004, 2011, 2012, 2016, 2017, 2018, 2019, 2020]",0,0,0,,
156,MAGNUM,Merlot Riserva “Passion Limited Edition” DOC 2...,173,,Merlot Riserva “Passion Limited Edition” DOC,,,"(+ 1,5 l,)",[2016],0,0,0,,
158,MAGNUM,Merlot “KUNST.STÜCK” RISERVA DOC 2018 Kellerei...,175,,Merlot “KUNST.STÜCK” RISERVA DOC Kellerei Kal...,,,"(+ 1,5 l,)",[2018],0,0,0,,


#### price

In [70]:
price_regex = r"\d+ ?€"

def get_price(x):
    price = re.findall(price_regex, x)
    if len(price) == 0:
        return 0
    return int(re.sub(" |€", "", price[0]))*100 # convert to cents


df2['price'] = df2['original_name'].apply(get_price)

# remove price from name
df2['name'] = df2['name'].apply(lambda x: re.sub(price_regex, "", x).strip())
df2['info'] = df2['info'].apply(lambda x: re.sub(price_regex, "", x).strip())

display(df2.loc[df2['price'].isna() | df2['price']==0])

df2.head()

,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
31,WHITE,Weissb. Vorberg Terlan,48,WHITE,Weissb. Vorberg Terlan,,,"(+ 0,75 l,)",0,0,0,0,,
32,WHITE,Weissb. Praesulis Gump Hof,49,WHITE,Weissb. Praesulis Gump Hof,,,"(+ 0,75 l,)",0,0,0,0,,
42,WHITE,Riesling Anaduron Falkensteiner,59,WHITE,Riesling Anaduron Falkensteiner,,,"(+ 0,75 l,)",0,0,0,0,,
67,RED,Vernatsch Muri + 1 l,84,RED,Vernatsch Muri,,,"(+ 1 l,)",0,0,0,0,,
73,RED,Blaub. Anrar Andrian,90,RED,Blaub. Anrar Andrian,,,"(+ 0,75 l,)",0,0,0,0,,
74,RED,Blaub. Baltasius schloss englar,91,RED,Blaub. Baltasius schloss englar,,,"(+ 0,75 l,)",0,0,0,0,,
79,RED,Pinot Nair Aton Elena Walch,96,RED,Pinot Nair Aton Elena Walch,,,"(+ 0,75 l,)",0,0,0,0,,
80,RED,"Pinot Nero Mason di mason manincor\n+ 1,5 l",97,RED,Pinot Nero Mason di mason manincor,,,"(+ 1,5 l,)",0,0,0,0,,
90,RED,merlot brenntal kurtatsch,107,RED,merlot brenntal kurtatsch,,,"(+ 0,75 l,)",0,0,0,0,,
91,RED,Merlot Siebeneich kellerei bozen,108,RED,Merlot Siebeneich kellerei bozen,,,"(+ 0,75 l,)",0,0,0,0,,


,section,original_name,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
0,SPARKLING,"PROSECCO CARMINA „LOGGIA“ + 0,75 l - 22 €",17,SPARKLING,PROSECCO CARMINA „LOGGIA“,,,"(+ 0,75 l,)",0,2200,0,0,,
1,SPARKLING,"Sekt Castel Monreale Rosé Rametz Südtirol + 0,...",18,SPARKLING,Sekt Castel Monreale Rosé Rametz Südtirol,,,"(+ 0,75 l,)",0,4500,0,0,,
2,SPARKLING,"Arunda Rosé Sekt - Südtirol + 0,75 l - 48 €",19,SPARKLING,Arunda Rosé Sekt - Südtirol,,,"(+ 0,75 l,)",0,4800,0,0,,
3,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Mer...,20,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Meran,,,"(+ 0,75 l, + 1,5 l)",0,4300,0,0,,
4,SPARKLING,"Comitissa Pas Dosé Riserva - Südtirol + 0,75 l...",21,SPARKLING,Comitissa Pas Dosé Riserva - Südtirol,,,"(+ 0,75 l, + 1,5 l)",0,4700,0,0,,


### Explode dataframe

In [71]:
from utils.size_mapping import size_to_name

df2_out = df2[VColumns.v2()]

# explode sizes
df2_out = df2_out.explode('size').reset_index(drop=True)

# convert size to standard names
display(df2_out.loc[df2_out['size'].isna()])
df2_out['size'] = df2_out['size'].apply(lambda x: re.sub("\+| |l", "", x))
df2_out['size'] = df2_out['size'].apply(lambda x: float(x.replace(",", ".")))
df2_out['size'] = df2_out['size'].map(size_to_name())


# explode vintage
df2_out = df2_out.explode('vintage').reset_index(drop=True)

display(df2_out.loc[df2_out['vintage'].isna()])



df2_out.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes


,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes


,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes
0,17,SPARKLING,PROSECCO CARMINA „LOGGIA“,,,BOTTLE,0,2200,0,0,,
1,18,SPARKLING,Sekt Castel Monreale Rosé Rametz Südtirol,,,BOTTLE,0,4500,0,0,,
2,19,SPARKLING,Arunda Rosé Sekt - Südtirol,,,BOTTLE,0,4800,0,0,,
3,20,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Meran,,,BOTTLE,0,4300,0,0,,
4,20,SPARKLING,Südtiroler Sekt Brut Riserva „36“ Kellerei Meran,,,MAGNUM,0,4300,0,0,,


# Merge files and write output

In [72]:
df1.head()

df_out = pd.concat([df1, df2_out], ignore_index=True)
df_out.to_csv("v2-cleaned.csv", index=False)